hypothesis: Grouping similar questions together will enhance the performance of the model in answering multiple-choice questions.
Rationale: The idea behind this hypothesis is that similar questions often share common themes, concepts, or structures. By grouping these questions together, the model can leverage the shared information to better understand the context and make more accurate predictions. This approach could potentially allow the model to recognize patterns or relationships between questions that it might miss when the questions are presented individually.
Proposed Method: To test this hypothesis, we could modify the training process to present batches of similar questions together. We could use various strategies to determine similarity, such as analyzing the content of the questions, the subject matter, or the structure of the questions. We could then compare the performance of the model when trained with these batches of similar questions versus its performance when trained with randomly ordered questions.
Expected Outcome: If the hypothesis is correct, we would expect to see an improvement in the model's performance when it is trained with batches of similar questions. This could manifest as an increase in accuracy, a decrease in errors, or other measurable improvements in the model's ability to answer multiple-choice questions.
Potential Challenges: One challenge in testing this hypothesis could be determining the best way to group similar questions together. Different strategies might yield different results, and it could require some experimentation to find the most effective approach. Additionally, it might be necessary to adjust the model's architecture or training process to effectively leverage the grouped questions.

In [ ]:
import pandas as pd
import requests
import re
from openai import OpenAI
import instructor
from instructor import patch
from pydantic import BaseModel
from typing import Dict

client = patch(OpenAI())
df = pd.read_csv('train.csv')
row = df.iloc[0]

test_item = row['prompt']  + ' ' + row['A'] + ' ' + row['B'] + ' ' + row['C'] + ' ' + row['D'] + ' ' + row['E'] + ' ' + row['answer']

client = instructor.patch(client)

class QuestionGroup(BaseModel):
    first: str
    second: str
    third: str
    
resp = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            "role": "user", 
            "content": f"Read the MCQ and give the top three terms you would use to search for an answer: {test_item}",
        },
    ],
    response_model=QuestionGroup
)
resp
print(resp)

def clean_string(s):
  # Remove everything inside parentheses
  s = re.sub(r'\([^()]*\)', '', s)
  
  # Remove all non-alphabetical characters except spaces
  s = re.sub(r'[^a-zA-Z ]', '', s)
  
  # Remove spaces from the end of the string
  s = s.rstrip()
  
  return s

search_term = clean_string(resp.first)
print(search_term)